In [1]:
import numpy as np 

In [2]:
def sortkeypicker(keynames):
    negate = set()
    for i, k in enumerate(keynames):
        if k[:1] == '-':
            keynames[i] = k[1:]
            negate.add(k[1:])
    def getit(adict):
       composite = [adict[k] for k in keynames]
       for i, (k, v) in enumerate(zip(keynames, composite)):
           if k in negate:
               composite[i] = -v
       return composite
    return getit

In [3]:
def paralel_check(v1,v2, trash_hold=0.00001):


    
    t_prod = np.cross(v1,v2)
    ar_2 = np.linalg.norm(t_prod)**2

    if ar_2<trash_hold:
        return True
    else:
        return False
    
# paralel_check(c1[0],c1[0], trash_hold=0.00001)



# v1_1= ca1+db1 => c= (v1_1-db1)/a1
# v1_2= ca2+db2 => v1_2= v1_1*a2/a1-db1*a2/a1+db2
# v1_2-v1_1*a2/a1=d(-b1*a2/a1+b2)
# b = (v1_2-v1_1*a2/a1)/(-b1*a2/a1+b2)
def decompose_to_basis(v, a,b):
    
    d = (v[1]-v[0]*a[1]/a[0])/(b[1]-b[0]*a[1]/a[0])
    c = (v[0]-d*b[0])/a[0]
    
#     print(v-c*a-d*b)
    return [c,d]

# decompose_to_basis(v=np.array([9,10]), a=np.array([2,3]),b=np.array([5,1]))

In [4]:
def point_in_cell(c, point):
    
    
    distance_to_close = 0
    s = decompose_to_basis(point, a=c[:,0],b=c[:,1])
    s1 = s[0]
    s2 = s[1]
    
    c_p = round(s1)*c[:,0]+round(s2)*c[:,1]
    distance_to_close = np.sqrt((c_p[0]-point[0])**2+(c_p[1]-point[1])**2)
    
#     print("s1:{}={}".format(round(s1),s1))
#     print("s2:{}={}".format(round(s2),s2))
#     print("distance_to_close:", distance_to_close)
    return distance_to_close

In [65]:
def get_match(cel1, cel2, exploring_range=10, tolerance=0.3, paralel_limit=0.00001):
    c = cel1
    cel1 = cel2
    cel2 = c
    c1 = (cel1[:,0],cel1[:,1])
    c2 = (cel2[:,0],cel2[:,1])
    
    candidates = []  #TODO: it can be done without list but for the beggining I preffer to keep it 
                     # in order to get a better view of the data
        
    for i in range(-exploring_range, exploring_range):
        for j in range(-exploring_range, exploring_range):
            point2 = i*c2[0]+j*c2[1]
            
            in_c1 = point_in_cell(cel1, point2)
            
            if in_c1<= tolerance:
                point_distance = point2[0]**2+point2[1]**2
                if point_distance!= 0:
                    candidates.append({"point2":point2, "in_c1":in_c1,"point_distance":point_distance}) 
                  
                    
    
    candidates = sorted(candidates, key=sortkeypicker(['point_distance', 'in_c1']), )
    
    
        
#     for c in candidates:
#         print(c)
        
    solution = [candidates[0], ]
    k=1
    while len(solution)<2 and len(candidates)>=k+2:
        k+=1
        close = candidates[k]
        min_v = close["point2"]
        if paralel_check(solution[0]["point2"], min_v,trash_hold=paralel_limit) ==False:
            solution.append(close)
        
        
    return solution
    

In [81]:
def extract_tB_strain(tB_strain, cel2):
    
    super_cell_ = np.dot(tB_strain,cel2).transpose()
    d1 = decompose_to_basis(super_cell_[0], a= cel2[:,0],b=cel2[:,1])
    d2 = decompose_to_basis(super_cell_[1], a= cel2[:,0],b=cel2[:,1])
    print(d1)
    d1_r = np.array([round(d1[0]),round(d1[1])])
    d2_r = np.array([round(d2[0]),round(d2[1])])
    almost_super = np.array([d1_r,d2_r]).transpose()
    tB = np.dot(almost_super ,np.linalg.inv(cel2))
    strain = np.dot(np.linalg.inv(tB),tB_strain)
    
    return tB, strain

In [87]:
a1 = [1,0]; a2=[0,1]
A  = np.transpose([a1,a2])

theta =  np.pi/4
R = [ [ np.cos(theta), -np.sin(theta)], [ np.sin(theta),  np.cos(theta)] ]

b1 = np.sqrt(2)*np.dot(R,a1)#[np.sqrt(2),0]#np.sqrt(2)*np.dot(R,a1);  #a1
b2 = np.sqrt(2)*np.dot(R,a2)#[0,np.sqrt(2)]#np.sqrt(2)*np.dot(R,a2) #a2
print(b2)
#b1 = [np.sqrt(2),0]; b2=[0,np.sqrt(2)]
B  = np.transpose([b1,b2])


cel1 = np.array(A)
cel2 = np.array(B)

print("cel1:\n", cel1)
print("cel1:\n", cel2)

[-1.  1.]
cel1:
 [[1 0]
 [0 1]]
cel1:
 [[ 1. -1.]
 [ 1.  1.]]


In [88]:
solution = get_match(cel1, cel2,exploring_range=10, tolerance=0.1, paralel_limit=0.00001)
print("super_cell:\n")
for s in solution:
    print(s)
    
supper_cell = np.array([solution[0]["point2"],solution[1]["point2"]]).transpose()
tA= np.dot(supper_cell, np.linalg.inv(cel1))
tB_strain= np.dot(supper_cell, np.linalg.inv(cel2))
tB, strain = extract_tB_strain(tB_strain, cel2)


v1: [-1 -1]
v2: [ 1 -1]
super_cell:

{'point2': array([-1, -1]), 'in_c1': 2.220446049250313e-16, 'point_distance': 2}
{'point2': array([ 1, -1]), 'in_c1': 2.220446049250313e-16, 'point_distance': 2}
[-0.9999999999999998, -1.6653345369377348e-16]


In [89]:
print("\ncel1\n",cel1)
print("cel2\n",cel2)
print("\n=====================\n")
print("\n _____\n tA:\n",tA)
print("det:",np.linalg.det(tA))
print("\n _____\n tB_strain:\n",tB_strain)
print("\n _____\n tB:\n",tB)
print("\n _____\n strain:\n",strain)
print("\n _____\n Super cell:\n",supper_cell)
print("\n=====================\n") 


cel1
 [[1 0]
 [0 1]]
cel2
 [[ 1. -1.]
 [ 1.  1.]]



 _____
 tA:
 [[-1.  1.]
 [-1. -1.]]
det: 2.0

 _____
 tB_strain:
 [[-1.00000000e+00  1.11022302e-16]
 [-1.11022302e-16 -1.00000000e+00]]

 _____
 tB:
 [[-0.5 -0.5]
 [ 0.5 -0.5]]

 _____
 strain:
 [[ 1. -1.]
 [ 1.  1.]]

 _____
 Super cell:
 [[-1  1]
 [-1 -1]]


